## Imports

In [15]:
import marvin
from typing import List, Dict, Tuple
from itertools import groupby
import ast
import pandas as pd
marvin.settings.llm_model='openai/gpt-4'

## Functions

In [12]:
@marvin.fn
def replace_prounouns(text: str) -> str:
    """Replaces pronouns in the text with the subject they refer to.
    """

@marvin.fn
def parse_into_statements(text: str) -> list[str]:
    """Parse the `text` into a list of strings containing each fact as a separate sentence. 
    Breaks up current sentences into each fact.
    """

@marvin.fn
def classify_answer_correctness(context_list: list[str], answer_list: list[str]) -> list[list[str]]:
    """
    Returns a list with the following list of strings:
    List of true positive statements that are semantically similar in both the `answer_list` and the `context_list`.
    List of false positive statements that are in the `answer_list` but not semantically similar to any statement in the `context_list`.
    List of false negative statements that are in the `context_list` but not semantically similar to any statement in the `answer_list`.
    """

def calculate_precision_recall(statement_list: List[str]) -> tuple[float, float]:
    """Calculates precision and recall from a list of statements."""
    tp = len(statement_list[0])
    fp = len(statement_list[1])
    fn = len(statement_list[2])

    print([tp, fp, fn])

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0

    return precision * 100, recall * 100


def process_data(data_list):
    results = []

    for data in data_list:
        context = data['context']
        answer = data['answer']

        context_replace_pronouns = replace_prounouns(context)
        answer_replace_pronouns = replace_prounouns(answer)

        context_list = parse_into_statements(context_replace_pronouns)
        answer_list = parse_into_statements(answer_replace_pronouns)

        classifications = classify_answer_correctness(context_list, answer_list)

        precision, recall = calculate_precision_recall(classifications)

        result_data = {
            'context': context,
            'answer': answer,
            'context_replace_pronouns': context_replace_pronouns,
            'answer_replace_pronouns': answer_replace_pronouns,
            'context_list': context_list,
            'answer_list': answer_list,
            'classifications': classifications,
            'precision': precision,
            'recall': recall
        }

        results.append(result_data)

    df = pd.DataFrame(results)
    return df

## Run on one pair of statements

In [3]:
context =  "The quick brown fox jumps over the rock because he's happy. He was born in 2005. The hedgehog was born in 2010, but she's even happier than him."
answer = "The quick brown fox was born in 2005, and the hedgehog in 2010. The quick brown fox is not as happy as the hedgehog"
context_replace_prounouns = replace_prounouns(context)
answer_replace_prounouns = replace_prounouns(answer )
context_list = parse_into_statements(context_replace_prounouns)
answer_list = parse_into_statements(answer_replace_prounouns)
classifications = classify_answer_correctness(context_list, answer_list)
precision, recall = calculate_precision_recall(classifications)
print("Precision: {:.2f}%".format(precision))
print("Recall: {:.2f}%".format(recall))


## Run on a list of dictionaries - return DF

In [19]:
data_list = [
    {
        'context': 'The quick brown fox jumps over the rock because he\'s happy. He was born in 2005. The hedgehog was born in 2010, but she\'s even happier than him.',
        'answer': 'The quick brown fox was born in 2005, and the hedgehog in 2010. The quick brown fox is not as happy as the hedgehog'
    },
    {
        'context': 'The sun is a star at the center of our solar system. It is about 93 million miles away from Earth. The sun is a hot ball of glowing gases that provides light and warmth to Earth.',
        'answer': 'The sun is a star located approximately 93 million miles from Earth. It is the source of light and heat for our planet. The sun is not a solid object, but rather a sphere of hot glowing gases.'
    },
    {
        'context': 'Birds are warm-blooded vertebrates that lay eggs and have feathers, wings, and beaks. There are over 10,000 species of birds worldwide. Some common bird species include sparrows, pigeons, and parrots.',
        'answer': 'Birds are a diverse group of animals with feathers and wings. They are warm-blooded egg-laying vertebrates. The number of bird species globally exceeds 10,000. Pigeons, parrots, and sparrows are among the most familiar bird types.'
    },
    {
        'context': 'The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It was constructed from 1887 to 1889 and stands at a height of 324 meters. The tower is named after Gustave Eiffel, whose company designed and built it.',
        'answer': 'The Eiffel Tower, found in Paris, France, is a lattice tower made of wrought iron. Built between 1887 and 1889, it reaches a height of 324 meters. Gustave Eiffel\'s company was responsible for the tower\'s design and construction, hence its name.'
    },
    {
        'context': 'The Great Wall of China is a series of fortifications and walls built across the historical northern borders of ancient Chinese states and Imperial China. The most well-known sections were built during the Ming dynasty, which ruled from 1368 to 1644.',
        'answer': 'The Great Wall of China, a series of walls and fortifications, was constructed along the northern borders of ancient Chinese states and Imperial China. The Ming dynasty, which lasted from 1368 to 1644, is responsible for the construction of the most famous sections of the wall.'
    }
]

result_df = process_data(data_list)

[['The quick brown fox was born in 2005.', 'The hedgehog was born in 2010.'], ['The quick brown fox is not as happy as the hedgehog.'], ['The quick brown fox jumps over the rock.', 'The quick brown fox is happy.', 'The hedgehog is even happier than the quick brown fox.']]
[2, 1, 3]
[['The sun is a star.', 'The sun is located approximately 93 million miles from Earth.', 'The sun is the source of light and heat for our planet.', 'The sun is a sphere of hot glowing gases.'], ['The sun is not a solid object.'], ['The sun is a hot ball of glowing gases.', 'The sun provides light and warmth to Earth.']]
[4, 1, 2]
[['Birds are warm-blooded egg-laying vertebrates.', 'The number of bird species globally exceeds 10,000.', 'Pigeons, parrots, and sparrows are among the most familiar bird types.'], ['Birds are a diverse group of animals with feathers and wings.'], ['Birds lay eggs.', 'Birds have feathers, wings, and beaks.', 'Some common bird species include sparrows.', 'Some common bird species in

In [24]:
result_df

,context,answer,context_replace_pronouns,answer_replace_pronouns,context_list,answer_list,classifications,precision,recall
0,The quick brown fox jumps over the rock becaus...,"The quick brown fox was born in 2005, and the ...",The quick brown fox jumps over the rock becaus...,"The quick brown fox was born in 2005, and the ...","[The quick brown fox jumps over the rock., The...","[The quick brown fox was born in 2005., The he...","[[The quick brown fox was born in 2005., The h...",66.666667,40.000000
1,The sun is a star at the center of our solar s...,The sun is a star located approximately 93 mil...,The sun is a star at the center of our solar s...,The sun is a star located approximately 93 mil...,[The sun is a star at the center of our solar ...,"[The sun is a star., The sun is located approx...","[[The sun is a star., The sun is located appro...",80.000000,66.666667
2,Birds are warm-blooded vertebrates that lay eg...,Birds are a diverse group of animals with feat...,Birds are warm-blooded vertebrates that lay eg...,Birds are a diverse group of animals with feat...,"[Birds are warm-blooded vertebrates., Birds la...",[Birds are a diverse group of animals with fea...,[[Birds are warm-blooded egg-laying vertebrate...,75.000000,37.500000
3,The Eiffel Tower is a wrought-iron lattice tow...,"The Eiffel Tower, found in Paris, France, is a...",The Eiffel Tower is a wrought-iron lattice tow...,"The Eiffel Tower, found in Paris, France, is a...",[The Eiffel Tower is a wrought-iron lattice to...,"[The Eiffel Tower is found in Paris, France., ...",[[The Eiffel Tower is a wrought-iron lattice t...,85.714286,85.714286
4,The Great Wall of China is a series of fortifi...,"The Great Wall of China, a series of walls and...",The Great Wall of China is a series of fortifi...,"The Great Wall of China, a series of walls and...",[The Great Wall of China is a series of fortif...,[The Great Wall of China is a series of walls ...,[[The Great Wall of China is a series of walls...,100.000000,60.000000
